In [5]:
!pip install gensim

     |████████████████████████████████| 26.6 MB 16.0 MB/s            
     |████████████████████████████████| 57 kB 938 kB/s             
You should consider upgrading via the '/usr/local/bin/python3.8 -m pip install --upgrade pip' command.


In [6]:
import pandas as pd
import numpy as np
from keras.preprocessing import text, sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Embedding
from keras.callbacks import EarlyStopping

# 네트워크 페이로드 데이터를 읽어온다.
df = pd.read_csv('../dataset/A-Track-clean.csv')

# 피처엔지니어링을 수행한다.
X = df['payload']
y = df['label_action']

# X를 정규화한다.
X = text.normalize(X)
import keras.preprocessing.text
X = keras.preprocessing.text.normalize(X)
# X를 단어 시퀀스로 변환한다.
maxlen = 256
X = sequence.pad_sequences(X, maxlen=maxlen)

# word2vec 모델을 생성한다.
w2v_model = Word2Vec(X, size=128, window=5, min_count=1)

# word2vec 모델을 사용하여 X를 word embedding으로 변환한다.
X = w2v_model.wv.transform(X)

# textCNN 모델을 생성한다.
model = Sequential()
model.add(Embedding(w2v_model.wv.size, 128, input_length=maxlen))
model.add(Conv1D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv1D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# 모델을 학습한다.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', patience=3)
model.fit(X, y, epochs=10, validation_split=0.2, callbacks=[es])

# 모델을 평가한다.
score = model.evaluate(X, y, verbose=0)
print('정확도:', score[1])


AttributeError: module 'keras.preprocessing.text' has no attribute 'normalize'

In [7]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from keras.preprocessing import text
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Embedding, Flatten
from keras.callbacks import EarlyStopping

# Load network payload data
df = pd.read_csv('../dataset/A-Track-clean.csv')

# Feature engineering
X = df['payload']
y = df['label_action']

# Text preprocessing and word sequence conversion
X = X.apply(text.text_to_word_sequence)
X = sequence.pad_sequences(X, maxlen=maxlen)

# Word2Vec model
w2v_model = Word2Vec(sentences=X, size=128, window=5, min_count=1)

# Word embedding using Word2Vec model
X = np.array([np.array([w2v_model.wv[word] for word in sentence]) for sentence in X])

# textCNN model
model = Sequential()
model.add(Embedding(input_dim=len(w2v_model.wv.vocab), output_dim=128, input_length=maxlen))
model.add(Conv1D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv1D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile and train the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', patience=3)
model.fit(X, y, epochs=10, validation_split=0.2, callbacks=[es])

# Evaluate the model
score = model.evaluate(X, y, verbose=0)
print('Accuracy:', score[1])


NameError: name 'maxlen' is not defined

In [4]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from keras.preprocessing import text
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Embedding, Flatten
from keras.callbacks import EarlyStopping

# Load network payload data
df = pd.read_csv('../dataset/A-Track-clean.csv')

# Feature engineering
X = df['payload']
y = df['label_action']

# Convert labels to numerical values
label_to_index = {label: i for i, label in enumerate(y.unique())}
y = y.map(label_to_index)
y = to_categorical(y, num_classes=len(label_to_index))

# Text preprocessing and word sequence conversion
X = X.apply(text.text_to_word_sequence)
maxlen = 256
X = sequence.pad_sequences(X, maxlen=maxlen)

# Word2Vec model
w2v_model = Word2Vec(sentences=X, size=128, window=5, min_count=1)

# Word embedding using Word2Vec model
X = np.array([np.array([w2v_model.wv[word] for word in sentence]) for sentence in X])

# CNN model
model = Sequential()
model.add(Embedding(input_dim=len(w2v_model.wv.vocab), output_dim=128, input_length=maxlen))
model.add(Conv1D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv1D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(label_to_index), activation='softmax'))  # Changed to softmax for multi-class

# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', patience=3)
model.fit(X, y, epochs=10, validation_split=0.2, callbacks=[es])

# Evaluate the model
score = model.evaluate(X, y, verbose=0)
print('Accuracy:', score[1])


ModuleNotFoundError: No module named 'gensim'

In [11]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from keras.preprocessing import text
from keras.preprocessing import sequence
# from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Embedding, Flatten
from keras.callbacks import EarlyStopping

# Load network payload data
df = pd.read_csv('../dataset/A-Track-clean.csv')

# Feature engineering
X = df['payload']
y = df['label_action']

# Convert labels to numerical values
label_to_index = {label: i for i, label in enumerate(y.unique())}
y = y.map(label_to_index)
y = to_categorical(y, num_classes=len(label_to_index))

tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
# Text preprocessing and word sequence conversion
X = X.apply(text.text_to_word_sequence)
maxlen = 256
X = sequence.pad_sequences(X, maxlen=maxlen)

# Word2Vec model
w2v_model = Word2Vec(sentences=X, size=128, window=5, min_count=1)

# Word embedding using Word2Vec model
X = np.array([np.array([w2v_model.wv[word] for word in sentence]) for sentence in X])

# CNN model
model = Sequential()
model.add(Embedding(input_dim=len(w2v_model.wv.vocab), output_dim=128, input_length=maxlen))
model.add(Conv1D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv1D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(label_to_index), activation='softmax'))  # Changed to softmax for multi-class

# Compile and train the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', patience=3)
model.fit(X, y, epochs=10, validation_split=0.2, callbacks=[es])

# Evaluate the model
score = model.evaluate(X, y, verbose=0)
print('Accuracy:', score[1])


AttributeError: 'list' object has no attribute 'apply'

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from keras.preprocessing import text
from keras.preprocessing import sequence
# from keras.utils import to_categorical
from keras.utils.np_utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, MaxPooling1D, Embedding, Flatten
from keras.callbacks import EarlyStopping


# Load network payload data
df = pd.read_csv('../dataset/A-Track-clean.csv')

# Feature engineering
X = df['payload']
y = df['label_action']

# Convert labels to numerical values and one-hot encode
label_to_index = {label: i for i, label in enumerate(y.unique())}
y = y.map(label_to_index)
y = to_categorical(y, num_classes=len(label_to_index))



tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)



maxlen = 256
X = sequence.pad_sequences(X, maxlen=maxlen)



w2v_model = Word2Vec(sentences=X, vector_size=128, window=5, min_count=1)
X = np.array([np.array([w2v_model.wv[word] for word in sentence]) for sentence in X])



model = Sequential()
model.add(Embedding(input_dim=len(w2v_model.wv.vocab), output_dim=128, input_length=maxlen))
model.add(Conv1D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Conv1D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(label_to_index), activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
es = EarlyStopping(monitor='val_loss', patience=3)
model.fit(X, y, epochs=10, validation_split=0.2, callbacks=[es])


Exception in thread Exception in thread Thread-7:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/threading.py", line 932, in _bootstrap_inner
Thread-6:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/threading.py", line 932, in _bootstrap_inner
        self.run()
  File "/usr/local/lib/python3.8/threading.py", line 870, in run
self.run()
  File "/usr/local/lib/python3.8/threading.py", line 870, in run
        self.run()
  File "/usr/local/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.8/site-packages/gensim/models/word2vec.py", line 1166, in _worker_loop
self._target(*self._args, **self._kwargs)    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.8/site-packages/gensim/models/word2vec.py", line 1166, in _wo